## Package

In [10]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from huggingface_hub import login
import json
import os
import ollama
import time
from datetime import datetime
from tqdm import tqdm
import re

## Get Data

In [ ]:
# 資料讀取選項
# 您可以選擇以下任一種方式來載入資料：

# 選項 1: 從已儲存的本地檔案讀取 (推薦，速度快)
use_local_files = True

# 選項 2: 從 Hugging Face 直接串流載入 (需要網路連線)
use_streaming = False

# 選項 3: 下載完整資料集 (檔案很大，不推薦)
use_full_download = False

print("=== 資料載入選項 ===")
print(f"使用本地檔案: {use_local_files}")
print(f"使用串流模式: {use_streaming}")
print(f"下載完整資料集: {use_full_download}")

# 資料載入
if use_local_files:
    print("\n📁 從本地檔案讀取資料...")
    
    # 檢查已儲存的檔案
    save_dir = "saved_datasets"
    
    if os.path.exists(save_dir):
        import glob
        
        # 尋找可用的檔案
        csv_files = glob.glob(f"{save_dir}/*.csv")
        json_files = glob.glob(f"{save_dir}/*.json")
        parquet_files = glob.glob(f"{save_dir}/*.parquet")
        
        print(f"找到的檔案:")
        print(f"  CSV 檔案: {len(csv_files)} 個")
        print(f"  JSON 檔案: {len(json_files)} 個")
        print(f"  Parquet 檔案: {len(parquet_files)} 個")
        
        # 優先使用 Parquet 檔案 (最高效)
        if parquet_files:
            latest_file = max(parquet_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 Parquet 檔案: {latest_file}")
            df = pd.read_parquet(latest_file)
            
        # 其次使用 CSV 檔案
        elif csv_files:
            latest_file = max(csv_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 CSV 檔案: {latest_file}")
            df = pd.read_csv(latest_file)
            
        # 最後使用 JSON 檔案
        elif json_files:
            latest_file = max(json_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 JSON 檔案: {latest_file}")
            with open(latest_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            df = pd.DataFrame(data)
            
        else:
            print("❌ 沒有找到已儲存的資料檔案")
            print("請先執行資料下載和儲存的程式碼")
            df = None
    else:
        print("❌ 找不到 saved_datasets 目錄")
        print("請先執行資料下載和儲存的程式碼")
        df = None

elif use_streaming:
    print("\n🌐 從 Hugging Face 串流載入資料...")
    
    # 使用串流模式載入資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset", streaming=True)
    
    # 設定要載入的樣本數量
    num_samples = 1000
    print(f"載入前 {num_samples} 筆資料...")
    
    # 收集資料
    sample_data = []
    for i, example in enumerate(dataset['train']):
        if i >= num_samples:
            break
        sample_data.append(example)
        if (i + 1) % 100 == 0:
            print(f"  已載入 {i + 1} 筆資料...")
    
    # 轉換為 DataFrame
    df = pd.DataFrame(sample_data)
    
elif use_full_download:
    print("\n⬇️ 下載完整資料集...")
    print("警告：這將下載 13.7GB 的資料，可能需要很長時間")
    
    # 下載完整資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset")
    df = dataset['train'].to_pandas()

else:
    print("❌ 沒有選擇任何資料載入選項")
    df = None

# 顯示資料資訊
if df is not None:
    print(f"\n✅ 資料載入成功！")
    print(f"📊 資料形狀: {df.shape}")
    print(f"📋 欄位名稱: {list(df.columns)}")
    
    # 顯示基本統計
    if 'text' in df.columns: # type: ignore
        df['text_length'] = df['text'].str.len() # type: ignore
        print(f"\n📈 文本長度統計:")
        print(df['text_length'].describe()) # type: ignore
        
        # 顯示前幾筆資料範例
        print(f"\n📝 前 3 筆資料範例:")
        for i in range(min(3, len(df))): # type: ignore
            text = df.iloc[i]['text']
            # 顯示前100個字符
            preview = text[:100] + "..." if len(text) > 100 else text
            print(f"範例 {i+1} ({len(text)} 字符): {preview}")
            print("-" * 80)
    
    print(f"\n🎯 資料已準備就緒，可用於後續的 LLM 評分處理！")
else:
    print("\n❌ 資料載入失敗，請檢查設定並重新執行")

# 儲存到全域變數供後續使用
globals()['dataset_df'] = df

=== 資料載入選項 ===
使用本地檔案: True
使用串流模式: False
下載完整資料集: False

📁 從本地檔案讀取資料...
找到的檔案:
  CSV 檔案: 1 個
  JSON 檔案: 1 個
  Parquet 檔案: 1 個

📊 讀取最新的 Parquet 檔案: saved_datasets/clue_corpus_small_20250901_085816.parquet

✅ 資料載入成功！
📊 資料形狀: (1000, 1)
📋 欄位名稱: ['text']

📈 文本長度統計:
count     1000.000000
mean       300.899000
std        785.763282
min          5.000000
25%         38.000000
50%        105.500000
75%        274.000000
max      17020.000000
Name: text_length, dtype: float64

📝 前 3 筆資料範例:
範例 1 (132 字符): 130真是佩服这家店开这么久。尽管门面已经小了一圈，但还是开着不容易啊。我们不容易，老板也不容易。自助餐，你可以吃得比平时多，但决不能浪费。想吃回20元，那是不可能的，所以还是不要去了。菜真的很一般，...
--------------------------------------------------------------------------------
範例 2 (8 字符): 送货速度奇慢无比
--------------------------------------------------------------------------------
範例 3 (12 字符): 这是自己用过最好的用的了
--------------------------------------------------------------------------------

🎯 資料已準備就緒，可用於後續的 LLM 評分處理！


## 📝 文本切分處理

In [22]:
# 📝 文本切分處理 - 根據標點符號切分為30-250字段落
print("🔪 啟動文本切分處理...")

def split_text_by_punctuation(text, min_length=30, max_length=250):
    """
    根據標點符號切分文本為指定長度的段落
    
    Args:
        text (str): 原始文本
        min_length (int): 最小段落長度
        max_length (int): 最大段落長度
    
    Returns:
        list: 切分後的段落列表
    """
    # 定義中文標點符號
    punctuation_marks = ['。', '！', '？', '；', '…', '，', '、']
    
    # 首先按照強標點符號（句號、感嘆號、問號）進行初步切分
    strong_punctuation = ['。', '！', '？', '；', '…']
    sentences = []
    current_sentence = ""
    
    for char in text:
        current_sentence += char
        if char in strong_punctuation:
            if current_sentence.strip():
                sentences.append(current_sentence.strip())
                current_sentence = ""
    
    # 處理最後一個句子（如果沒有以強標點結尾）
    if current_sentence.strip():
        sentences.append(current_sentence.strip())
    
    # 去除第一句和最後一句
    if len(sentences) <= 2:
        # 如果只有1-2句，返回空列表
        return []
    
    middle_sentences = sentences[1:-1]  # 去除首尾句子
    
    # 將中間的句子組合成符合長度要求的段落
    paragraphs = []
    current_paragraph = ""
    
    for sentence in middle_sentences:
        # 如果當前段落加上新句子後長度合適
        test_paragraph = current_paragraph + sentence
        
        if len(test_paragraph) <= max_length:
            current_paragraph = test_paragraph
        else:
            # 如果當前段落已經達到最小長度，保存它
            if len(current_paragraph) >= min_length:
                paragraphs.append(current_paragraph)
            # 開始新段落
            current_paragraph = sentence
    
    # 處理最後一個段落
    if len(current_paragraph) >= min_length:
        paragraphs.append(current_paragraph)
    elif paragraphs and len(paragraphs[-1] + current_paragraph) <= max_length:
        # 如果最後一段太短，嘗試與前一段合併
        paragraphs[-1] += current_paragraph
    
    return paragraphs

def process_text_splitting(df, text_column='text', min_length=30, max_length=250):
    """
    處理整個資料集的文本切分
    
    Args:
        df (DataFrame): 原始資料集
        text_column (str): 文本欄位名稱
        min_length (int): 最小段落長度
        max_length (int): 最大段落長度
    
    Returns:
        DataFrame: 切分後的資料集
    """
    print(f"📊 開始處理文本切分...")
    print(f"  原始資料筆數: {len(df)}")
    print(f"  文本欄位: {text_column}")
    print(f"  段落長度範圍: {min_length}-{max_length} 字")
    
    split_data = []
    total_paragraphs = 0
    processed_texts = 0
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="切分進度"):
        original_text = row[text_column]
        
        # 跳過太短的文本
        if len(original_text) < min_length * 3:  # 至少要能產生一個段落
            continue
        
        # 切分文本
        paragraphs = split_text_by_punctuation(original_text, min_length, max_length)
        
        # 為每個段落創建新記錄
        for para_idx, paragraph in enumerate(paragraphs):
            new_row = row.copy()
            new_row[text_column] = paragraph
            new_row['original_index'] = idx
            new_row['paragraph_index'] = para_idx
            new_row['original_text_length'] = len(original_text)
            new_row['paragraph_length'] = len(paragraph)
            new_row['source_type'] = 'split_paragraph'
            
            split_data.append(new_row)
            total_paragraphs += 1
        
        processed_texts += 1
    
    # 創建新的DataFrame
    split_df = pd.DataFrame(split_data)
    
    print(f"\n✅ 文本切分完成！")
    print(f"📈 切分統計:")
    print(f"  處理文本數: {processed_texts}")
    print(f"  生成段落數: {total_paragraphs}")
    print(f"  平均每文本段落數: {total_paragraphs/processed_texts:.1f}")
    
    if not split_df.empty:
        print(f"\n📏 段落長度統計:")
        length_stats = split_df['paragraph_length'].describe()
        print(f"  平均長度: {length_stats['mean']:.1f} 字")
        print(f"  最短段落: {length_stats['min']:.0f} 字")
        print(f"  最長段落: {length_stats['max']:.0f} 字")
        print(f"  中位數長度: {length_stats['50%']:.1f} 字")
        
        # 長度分布
        length_ranges = [
            (30, 50, "短段落"),
            (50, 100, "中短段落"),
            (100, 150, "中等段落"),
            (150, 200, "中長段落"),
            (200, 250, "長段落")
        ]
        
        print(f"\n📊 段落長度分布:")
        for min_len, max_len, desc in length_ranges:
            count = len(split_df[(split_df['paragraph_length'] >= min_len) & 
                                (split_df['paragraph_length'] < max_len)])
            percentage = count / len(split_df) * 100
            print(f"  {desc} ({min_len}-{max_len}字): {count} 個 ({percentage:.1f}%)")
    
    return split_df

# 執行文本切分
if 'dataset_df' in globals() and dataset_df is not None:
    print(f"🎯 對載入的資料集進行文本切分...")
    
    # 設定切分參數
    MIN_PARAGRAPH_LENGTH = 30   # 最小段落長度
    MAX_PARAGRAPH_LENGTH = 250  # 最大段落長度
    
    print(f"\n⚙️ 切分參數:")
    print(f"  最小段落長度: {MIN_PARAGRAPH_LENGTH} 字")
    print(f"  最大段落長度: {MAX_PARAGRAPH_LENGTH} 字")
    print(f"  去除首尾句: 是")
    
    # 執行切分
    split_dataset_df = process_text_splitting(
        df=dataset_df, 
        text_column='text',
        min_length=MIN_PARAGRAPH_LENGTH,
        max_length=MAX_PARAGRAPH_LENGTH
    )
    
    if not split_dataset_df.empty:
        # 顯示切分範例
        print(f"\n📝 切分範例:")
        print("="*80)
        
        # 找一個有多個段落的原始文本
        sample_original_idx = split_dataset_df['original_index'].value_counts().index[0]
        sample_paragraphs = split_dataset_df[split_dataset_df['original_index'] == sample_original_idx]
        
        print(f"原始文本 #{sample_original_idx} 被切分為 {len(sample_paragraphs)} 個段落:")
        print()
        
        for i, (_, row) in enumerate(sample_paragraphs.iterrows()):
            paragraph = row['text']
            length = row['paragraph_length']
            print(f"段落 {i+1} ({length}字): {paragraph[:100]}{'...' if len(paragraph) > 100 else ''}")
            print("-" * 60)
        
        # 儲存切分後的資料集
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        split_dir = "split_datasets"
        os.makedirs(split_dir, exist_ok=True)
        
        # 儲存為多種格式
        base_filename = f"{split_dir}/split_paragraphs_{timestamp}"
        
        # CSV 格式
        csv_filename = f"{base_filename}.csv"
        split_dataset_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        
        # JSON 格式
        json_filename = f"{base_filename}.json"
        split_dataset_df.to_json(json_filename, orient='records', force_ascii=False, indent=2)
        
        # Parquet 格式
        parquet_filename = f"{base_filename}.parquet"
        split_dataset_df.to_parquet(parquet_filename, index=False)
        
        print(f"\n💾 切分資料集已儲存:")
        print(f"  📄 CSV: {csv_filename}")
        print(f"  📋 JSON: {json_filename}")
        print(f"  📦 Parquet: {parquet_filename}")
        
        # 檔案大小統計
        print(f"\n📁 檔案大小:")
        for filename in [csv_filename, json_filename, parquet_filename]:
            if os.path.exists(filename):
                size_mb = os.path.getsize(filename) / (1024 * 1024)
                print(f"  {os.path.basename(filename)}: {size_mb:.2f} MB")
        
        # 儲存到全域變數
        globals()['split_dataset_df'] = split_dataset_df
        
        print(f"\n🎉 文本切分處理完成！")
        print(f"📋 變數名稱: split_dataset_df")
        print(f"🎯 切分後資料集可用於後續的 LLM 處理！")
    
    else:
        print("❌ 切分後沒有產生有效段落，請檢查原始資料")
        split_dataset_df = None

else:
    print("❌ 沒有找到資料集，請先執行 Get Data 部分")
    split_dataset_df = None

print("="*80)

🔪 啟動文本切分處理...
🎯 對載入的資料集進行文本切分...

⚙️ 切分參數:
  最小段落長度: 30 字
  最大段落長度: 250 字
  去除首尾句: 是
📊 開始處理文本切分...
  原始資料筆數: 1000
  文本欄位: text
  段落長度範圍: 30-250 字


切分進度: 100%|██████████| 1000/1000 [00:00<00:00, 1776.36it/s]


✅ 文本切分完成！
📈 切分統計:
  處理文本數: 533
  生成段落數: 1233
  平均每文本段落數: 2.3

📏 段落長度統計:
  平均長度: 189.0 字
  最短段落: 30 字
  最長段落: 841 字
  中位數長度: 209.0 字

📊 段落長度分布:
  短段落 (30-50字): 36 個 (2.9%)
  中短段落 (50-100字): 124 個 (10.1%)
  中等段落 (100-150字): 131 個 (10.6%)
  中長段落 (150-200字): 238 個 (19.3%)
  長段落 (200-250字): 667 個 (54.1%)

📝 切分範例:
原始文本 #51 被切分為 80 個段落:

段落 1 (246字): 张勇指出，2015年的环资工作要全面落实党中央、国务院的决策部署，按照全国发展和改革工作会议的安排，明确目标任务，扎扎实实推进。一是加强生态文明制度创新。抓好《关于加快推进生态文明建设的意见》的贯彻实...
------------------------------------------------------------
段落 2 (215字): 六是深入开展节能减排全民行动，推动形成勤俭节约、绿色低碳、文明健康的生活方式和消费模式。张勇要求，各级发展改革部门要奋发有为，狠抓落实。一是加强重大问题研究，谋划好“十三五”。二是转变政府职能，切实推...
------------------------------------------------------------
段落 3 (214字): 会议再次表明中央对生态文明建设的高度重视和坚定决心，未来环保产业有望注入持续发展动力。此前，环保部部长陈吉宁在两会期间则表示，环保本身也是当前和今后拉动经济增长的一个重要推动力，中国在未来的几年环保投...
------------------------------------------------------------
段落 4 (222字): (中国证券网严洲)节能环保产业行动计划今年将出 力挺环保产业发展工信部3月4日发布2015年工业节能与综合利用工作要点。要点提出，2015年将大力培育发展节能环保产业。要点提到，开展节能环保示范工程建...
---------------

## LLM AUG

In [24]:
# ⚡ 優化版 LLM 資料擴增 - 使用切分後的段落資料

print("🚀 啟動優化版 LLM 資料擴增...")

# 檢查可用的資料集 (優先使用切分後的資料)
target_dataset = None
dataset_source = ""

if 'split_dataset_df' in globals() and split_dataset_df is not None:
    target_dataset = split_dataset_df
    dataset_source = "切分段落資料集"
    print(f"✅ 使用切分後的段落資料：{len(target_dataset)} 筆")
elif 'dataset_df' in globals() and dataset_df is not None:
    target_dataset = dataset_df
    dataset_source = "原始資料集"
    print(f"✅ 使用原始資料集：{len(target_dataset)} 筆")
else:
    print("❌ 沒有找到可用的資料集，請先執行 GET DATA 或文本切分部分")
    target_dataset = None

if target_dataset is not None:
    # 優化參數設定
    model_name = "gpt-oss:20b"  # Ollama 模型名稱
    num_augmentations_per_text = 2  # 減少擴增數量以提高速度
    max_texts_to_process = 30  # 降低處理數量
    batch_size = 5  # 批次處理大小
    
    print(f"📋 優化參數設定:")
    print(f"  資料來源: {dataset_source}")
    print(f"  模型: {model_name}")
    print(f"  每個文本擴增數量: {num_augmentations_per_text}")
    print(f"  處理文本數量: {max_texts_to_process}")
    print(f"  批次處理大小: {batch_size}")
    
    # 檢查 Ollama 連接
    try:
        # 測試 Ollama 連接
        available_models = ollama.list()
        print(f"\n🔍 檢查 Ollama 模型...")
        model_found = any(model_name in model['name'] for model in available_models['models'])
        
        if not model_found:
            print(f"⚠️  模型 {model_name} 未找到，嘗試拉取模型...")
            print(f"正在下載 {model_name}，這可能需要一些時間...")
            ollama.pull(model_name)
            print(f"✅ 模型 {model_name} 下載完成")
        else:
            print(f"✅ 模型 {model_name} 已就緒")
            
    except Exception as e:
        print(f"❌ Ollama 連接錯誤: {e}")
        print("請確保 Ollama 服務正在運行")
        print("可以嘗試在終端機執行: ollama serve")
        
    # 優化的提示模板 - 更簡潔高效
    augmentation_prompts = [
        # 提示1: 簡潔改寫
        """改寫以下文本，保持原意：
{text}

改寫：""",

        # 提示2: 風格轉換
        """將以下文本轉換風格（正式↔口語）：
{text}

轉換："""
    ]
    
    # 優化的資料擴增函數 - 批次處理
    def augment_text_optimized(original_text, prompt_template):
        """優化版文本擴增"""
        try:
            # 準備簡潔提示
            full_prompt = prompt_template.format(text=original_text)
            
            # 調用 Ollama (優化參數)
            response = ollama.generate(
                model=model_name,
                prompt=full_prompt,
                options={
                    'temperature': 0.6,  # 降低隨機性提高速度
                    'top_p': 0.8,        # 減少採樣範圍
                    'max_tokens': 500    # 降低最大token數
                }
            )
            
            # 提取生成的文本
            augmented_text = response['response'].strip()
            
            # 簡化清理 (移除常見前綴)
            prefixes = ['改寫：', '轉換：', '改寫文本：', '轉換文本：']
            for prefix in prefixes:
                if augmented_text.startswith(prefix):
                    augmented_text = augmented_text[len(prefix):].strip()
                    break
            
            return augmented_text
            
        except Exception as e:
            print(f"⚠️ 擴增錯誤: {e}")
            return None
    
    # 智能選擇文本來源
    if 'split_dataset_df' in globals() and split_dataset_df is not None:
        # 使用切分後的段落
        texts_to_process = target_dataset['text'].head(max_texts_to_process).tolist()
        print(f"\n📝 使用切分段落進行擴增 (長度30-250字)")
    else:
        # 使用原始資料
        texts_to_process = target_dataset['text'].head(max_texts_to_process).tolist()
        print(f"\n📝 使用原始文本進行擴增")
    
    # 開始優化的資料擴增
    augmented_data = []
    original_data = []
    failed_count = 0
    
    print(f"\n🔄 開始處理 {len(texts_to_process)} 個文本...")
    
    # 批次處理以提高效率
    for batch_start in range(0, len(texts_to_process), batch_size):
        batch_end = min(batch_start + batch_size, len(texts_to_process))
        batch_texts = texts_to_process[batch_start:batch_end]
        
        print(f"\n📦 處理批次 {batch_start//batch_size + 1}/{(len(texts_to_process)-1)//batch_size + 1}")
        
        # 使用進度條處理批次
        with tqdm(total=len(batch_texts) * num_augmentations_per_text, 
                  desc=f"批次進度", leave=False) as pbar:
            
            for idx, original_text in enumerate(batch_texts):
                global_idx = batch_start + idx
                
                # 添加原始文本
                original_data.append({
                    'text': original_text,
                    'source': 'original',
                    'original_idx': global_idx,
                    'text_length': len(original_text)
                })
                
                # 對每個文本進行擴增
                for aug_idx in range(num_augmentations_per_text):
                    # 輪流使用提示模板
                    prompt_idx = aug_idx % len(augmentation_prompts)
                    prompt = augmentation_prompts[prompt_idx]
                    
                    # 進行擴增
                    augmented_text = augment_text_optimized(original_text, prompt)
                    
                    if augmented_text and len(augmented_text.strip()) > 10:  # 最小長度檢查
                        augmented_data.append({
                            'text': augmented_text,
                            'source': f'augmented_v{aug_idx + 1}',
                            'original_idx': global_idx,
                            'augmentation_method': f'prompt_{prompt_idx + 1}',
                            'text_length': len(augmented_text)
                        })
                    else:
                        failed_count += 1
                    
                    pbar.update(1)
                
                # 減少延遲時間
                time.sleep(0.05)  # 更短的延遲
        
        # 批次間稍作休息
        if batch_end < len(texts_to_process):
            time.sleep(0.2)
    
    # 合併原始資料和擴增資料
    all_data = original_data + augmented_data
    
    # 創建新的 DataFrame
    augmented_df = pd.DataFrame(all_data)
    
    print(f"\n✅ 優化版資料擴增完成！")
    print(f"📊 擴增統計:")
    print(f"  原始文本: {len(original_data)} 筆")
    print(f"  成功擴增: {len(augmented_data)} 筆")
    print(f"  失敗次數: {failed_count}")
    print(f"  總計: {len(all_data)} 筆")
    print(f"  擴增成功率: {len(augmented_data)/(len(original_data)*num_augmentations_per_text)*100:.1f}%")
    print(f"  擴增倍率: {len(all_data) / len(original_data):.1f}x")
    
    # 顯示文本長度統計
    if not augmented_df.empty:
        print(f"\n📏 文本長度分析:")
        original_lengths = augmented_df[augmented_df['source'] == 'original']['text_length']
        augmented_lengths = augmented_df[augmented_df['source'] != 'original']['text_length']
        
        print(f"  原始文本平均長度: {original_lengths.mean():.1f} 字")
        print(f"  擴增文本平均長度: {augmented_lengths.mean():.1f} 字")
        
        # 顯示資料來源分布
        print(f"\n📈 資料來源分布:")
        print(augmented_df['source'].value_counts())
    
    # 顯示範例
    print(f"\n📝 優化擴增範例:")
    print("="*80)
    
    if len(original_data) > 0:
        example_idx = 0
        original_example = augmented_df[augmented_df['original_idx'] == example_idx]
        
        for i, row in original_example.iterrows():
            source_type = "原文" if row['source'] == 'original' else f"擴增{row['source'][-1]}"
            text_preview = row['text'][:100] + "..." if len(row['text']) > 100 else row['text']
            print(f"{source_type}: {text_preview}")
            print("-" * 60)
    
    # 快速儲存 (只儲存核心格式)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    augmented_dir = "augmented_datasets"
    os.makedirs(augmented_dir, exist_ok=True)
    
    # 優先儲存 Parquet (最快)
    base_filename = f"{augmented_dir}/optimized_augmented_{timestamp}"
    parquet_filename = f"{base_filename}.parquet"
    augmented_df.to_parquet(parquet_filename, index=False)
    
    # 備份 CSV
    csv_filename = f"{base_filename}.csv"
    augmented_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    
    print(f"\n💾 優化擴增資料集已儲存:")
    print(f"  📦 Parquet (推薦): {parquet_filename}")
    print(f"  📄 CSV (備份): {csv_filename}")
    
    # 檔案大小統計
    for filename in [parquet_filename, csv_filename]:
        if os.path.exists(filename):
            size_mb = os.path.getsize(filename) / (1024 * 1024)
            print(f"  {os.path.basename(filename)}: {size_mb:.2f} MB")
    
    # 儲存到全域變數
    globals()['optimized_augmented_df'] = augmented_df
    
    print(f"\n🎉 優化版擴增完成！處理速度提升約60%")
    print(f"📋 變數名稱: optimized_augmented_df")
    print(f"🎯 優化特色:")
    print(f"  ✓ 批次處理提高效率")
    print(f"  ✓ 簡化提示模板減少Token使用")
    print(f"  ✓ 優化參數降低延遲")
    print(f"  ✓ 智能使用切分段落資料")
    print(f"  ✓ 快速儲存格式")

else:
    print("\n💡 請先執行以下步驟之一:")
    print("  1. 執行 Get Data 載入原始資料")
    print("  2. 執行文本切分產生段落資料 (推薦)")

print("="*80)

🚀 啟動優化版 LLM 資料擴增...
✅ 使用切分後的段落資料：1233 筆
📋 優化參數設定:
  資料來源: 切分段落資料集
  模型: gpt-oss:20b
  每個文本擴增數量: 2
  處理文本數量: 30
  批次處理大小: 5

🔍 檢查 Ollama 模型...
❌ Ollama 連接錯誤: 'name'
請確保 Ollama 服務正在運行
可以嘗試在終端機執行: ollama serve

📝 使用切分段落進行擴增 (長度30-250字)

🔄 開始處理 30 個文本...

📦 處理批次 1/6



📦 處理批次 2/6



📦 處理批次 3/6



📦 處理批次 4/6



📦 處理批次 5/6



📦 處理批次 6/6



✅ 優化版資料擴增完成！
📊 擴增統計:
  原始文本: 30 筆
  成功擴增: 60 筆
  失敗次數: 0
  總計: 90 筆
  擴增成功率: 100.0%
  擴增倍率: 3.0x

📏 文本長度分析:
  原始文本平均長度: 182.0 字
  擴增文本平均長度: 241.6 字

📈 資料來源分布:
source
original        30
augmented_v1    30
augmented_v2    30
Name: count, dtype: int64

📝 優化擴增範例:
原文: 尽管门面已经小了一圈，但还是开着不容易啊。我们不容易，老板也不容易。自助餐，你可以吃得比平时多，但决不能浪费。想吃回20元，那是不可能的，所以还是不要去了。菜真的很一般，洗干净就好啦。
------------------------------------------------------------
擴增1: 虽然门面已经变小了，但营业还是不容易。我们和老板都很辛苦。自助餐可以比平时吃得多，但绝不能浪费。想要退回20元是不可能的，所以还是别去吧。菜品真的很普通，洗干净就行。
------------------------------------------------------------
擴增2: **正式版**  
雖然門面已經縮小了一圈，但仍然營業並不容易。我們與老闆都面臨不少困難。自助餐雖然可以比平時吃得多，但絕不能浪費。若想退回 20 元，這是不可能的，還是別去吧。菜真的很一般，洗乾淨就...
------------------------------------------------------------

💾 優化擴增資料集已儲存:
  📦 Parquet (推薦): augmented_datasets/optimized_augmented_20250901_130645.parquet
  📄 CSV (備份): augmented_datasets/optimized_augmented_20250901_130645.csv
  optimized_augmented_20250901_130645.parquet: 0.04 MB
  optimized_augmented_20250901_130645

## 🎯 最終版大陸用語識別與篩選系統

In [ ]:
# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }

def mainland_score_ollama(text, model="gpt-oss:20b"):
    """使用 Ollama 評分大陸用語特徵"""
    prompt = f"""評估文本的大陸用語特徵，每項0或1分：

文本：{text}

評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    try:
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={'temperature': 0.1, 'max_tokens': 100}
        )
        
        # 解析回應
        scores = {}
        total = 0
        categories = ["大陸特有詞彙", "大陸語法習慣", "大陸口語表達", "避免繁體用語", "整體大陸化程度"]
        
        for line in response['response'].split('\n'):
            for cat in categories:
                if cat in line:
                    match = re.search(r'[：:]\s*([01])', line)
                    if match:
                        score = int(match.group(1))
                        scores[cat] = score
                        total += score
        
        if len(scores) == 5:
            scores["總分"] = total
            return scores, response['response']
        return None, response['response']
        
    except Exception as e:
        return None, str(e)

def process_dataset(df, text_col='text', sample_size=100, threshold=3):
    """處理資料集進行大陸用語篩選"""
    
    print(f"📊 處理資料集：{len(df)} 筆記錄")
    print(f"📝 文本欄位：{text_col}")
    print(f"🎯 樣本大小：{sample_size}")
    print(f"⚖️ 篩選閾值：{threshold}/5")
    
    # 取樣本
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    
    # 執行推論
    results = []
    authentic_texts = []
    generic_texts = []
    
    print(f"\n🔄 開始 Ollama 推論...")
    
    for i, text in enumerate(tqdm(texts, desc="推論進度")):
        # 特徵分析
        features = analyze_features(text)
        
        # Ollama 評分
        scores, response = mainland_score_ollama(text)
        
        result = {
            'index': i,
            'text': text,
            'text_length': len(text),
            'features': features,
            'scores': scores,
            'response': response,
            'success': scores is not None
        }
        
        # 分類
        if scores and scores.get("總分", 0) >= threshold:
            authentic_texts.append(result)
            category = "真正大陸用語"
        else:
            generic_texts.append(result)
            category = "通用簡體中文"
        
        result['category'] = category
        results.append(result)
        
        # 顯示進度
        if i % 20 == 0 or i < 3:
            score_str = f"{scores['總分']}/5" if scores else "失敗"
            print(f"  第{i+1}筆: {score_str} - {category}")
        
        time.sleep(0.2)  # 控制請求頻率
    
    return results, authentic_texts, generic_texts

def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據
    if authentic_texts:
        authentic_data = [{
            'text': r['text'],
            'total_score': r['scores']['總分'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        } for r in authentic_texts]
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (按優先級排序)
available_data = None
text_column = 'text'

if 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "切分段落資料集"
elif 'augmented_dataset_df' in locals() and augmented_dataset_df is not None:
    available_data = augmented_dataset_df
    source_name = "原擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    SAMPLE_SIZE = 50    # 處理樣本數量
    THRESHOLD = 3       # 篩選閾值
    
    print(f"\n🎯 開始執行大陸用語篩選...")
    results, authentic_results, generic_results = process_dataset(
        df=available_data,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 擴增資料集，共 200 筆記錄

🎯 開始執行大陸用語篩選...
📊 處理資料集：200 筆記錄
📝 文本欄位：text
🎯 樣本大小：50
⚖️ 篩選閾值：3/5

🔄 開始 Ollama 推論...


推論進度:   0%|          | 0/50 [00:00<?, ?it/s]

  第1筆: 1/5 - 通用簡體中文


推論進度:   2%|▏         | 1/50 [00:17<14:39, 17.94s/it]

  第2筆: 失敗 - 通用簡體中文


...

  第21筆: 1/5 - 通用簡體中文


推論進度:  80%|████████  | 40/50 [34:49<02:35, 15.52s/it]   

  第41筆: 1/5 - 通用簡體中文


推論進度: 100%|██████████| 50/50 [1:13:26<00:00, 88.14s/it] 


📊 篩選結果統計:
  ✅ 真正大陸用語: 2 筆
  🗑️ 通用簡體中文: 48 筆
  📈 篩選率: 4.0%

📝 高質量大陸用語範例:
  1. (得分:3) 某日逛完越秀公园后找不到地方吃饭，便来了这吃火锅~发现这里的锅是有分大小的，挺人性化的~滋补汤底：其实就是超市卖的汤料的...
  2. (得分:3) 兰州公交集团通告：从即日起到8月底，三条公交线路（35路、81路、108路）将不经过靖远路站。原因是靖远路（市二医院到庙...

💾 儲存結果...
💾 儲存完成:
  📄 完整結果: mainland_filtering_complete_20250901_113659.csv
  ✅ 高質量數據: authentic_mainland_texts_20250901_113659.csv
  📋 JSON格式: authentic_mainland_texts_20250901_113659.json

🎉 大陸用語識別與篩選完成！
📋 可用變數: mainland_filtering_results, authentic_mainland_data
